In [ ]:
import requests
import json
import csv

URL = 'http://www.thebluealliance.com/api/v2'
EVENT_URL = '/event/'
MATCHES_URL = '/matches'
EVENT_ID = '2017mawor' # Modify for specific event

HEADER_KEY = 'X-TBA-App-Id'
HEADER_VAL = 'frc:test-queuer:v01'

QUALIFICATION_ID = 'qm'
MATCHES_OUT_MAX = 4

output_file = 'schedule.tsv'

In [ ]:
def get_team_str(team):
    if team['next_match'] is not None:
        return '{} ({})'.format(team['team'], team['next_match'])
    else:
        return team['team']

class Match:
    match_num = -1
    teams = None
    
    def __init__(self, match_num, blue_teams, red_teams):
        self.match_num = match_num
        self.teams = dict()
        
        for i in range(len(blue_teams)):
            self.teams['B{}'.format(i + 1)] = {'team': blue_teams[i][3:], 'next_match': None}
        
        for i in range(len(red_teams)):
            self.teams['R{}'.format(i + 1)] = {'team': red_teams[i][3:], 'next_match': None}
    
    def __str__(self):
        return '{:2d}: Blue:{},{},{} | Red:{},{},{}'.format(self.match_num,
                                            get_team_str(self.teams['B1']),
                                            get_team_str(self.teams['B2']),
                                            get_team_str(self.teams['B3']),
                                            get_team_str(self.teams['R1']),
                                            get_team_str(self.teams['R2']),
                                            get_team_str(self.teams['R3']))

    def __repr(self):
        return '{:2d}: Blue:{},{},{} | Red:{},{},{}'.format(self.match_num,
                                            get_team_str(self.teams['B1']),
                                            get_team_str(self.teams['B2']),
                                            get_team_str(self.teams['B3']),
                                            get_team_str(self.teams['R1']),
                                            get_team_str(self.teams['R2']),
                                            get_team_str(self.teams['R3']))

In [ ]:
def get_event_matches(event_id):
    request_url = URL + EVENT_URL + event_id + MATCHES_URL
    response = requests.get(request_url, headers={HEADER_KEY: HEADER_VAL})
    json_response = response.json()
    
    return json_response

json_response = get_event_matches(EVENT_ID)

In [ ]:
matches = []

for match in json_response:
    if match['comp_level'] == QUALIFICATION_ID:
        matches.append(Match(match['match_number'], match['alliances']['blue']['teams'], match['alliances']['red']['teams']))
        
matches = sorted(matches, key=lambda x : x.match_num, reverse=False)

matches_count = len(matches)

def search_alliance_teams(all_matches, total_match_count, current_match):
    for team in current_match.teams.values():
        for next_index in range(1, MATCHES_OUT_MAX + 1):
            next_match_index = index + next_index
            if next_match_index >= total_match_count:
                continue
            
            next_match = all_matches[next_match_index]
            
            update_back_to_back_from_next_alliance(team, next_match, next_index)

def update_back_to_back_from_next_alliance(current_team, next_match, match_index_out):
    for next_team_key in next_match.teams.keys():
        next_team_val = next_match.teams[next_team_key]
        if current_team['team'] == next_team_val['team']:
            current_team['next_match'] = 'M{}:{}:+{}'.format(next_match.match_num, next_team_key, match_index_out)
            
for index in range(matches_count):
    match = matches[index]
    search_alliance_teams(matches, matches_count, match)

for match in matches:
    print match

In [ ]:
with open(output_file, "wb") as tsv_file:
    writer = csv.writer(tsv_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_NONE)
    writer.writerow(['Match Number', 'Blue 1', 'Blue 2', 'Blue 3', 'Red 1', 'Red 2', 'Red 3'])
    
    for match in matches:
        writer.writerow([match.match_num, get_team_str(match.teams['B1']), get_team_str(match.teams['B2']),
                         get_team_str(match.teams['B3']), get_team_str(match.teams['R1']), 
                         get_team_str(match.teams['R2']), get_team_str(match.teams['R3'])])
    print 'Wrote output to {}'.format(output_file)